In [13]:
import sys
sys.path.append(r'C:\Users\Usuário\Documents\Github\Forex-Forecast\pricelevels')
from cluster import ZigZagClusterLevels
from visualization.levels_with_zigzag import plot_with_pivots
import pandas as pd
import MetaTrader5 as mt5
import plotly 
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()



In [108]:
ativo = 'EURUSD'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M5, 0, 1000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})
df

,Data,Open,High,Low,Close,Vol
0,2023-01-10 12:35:00,1.07374,1.07392,1.07357,1.07373,0
1,2023-01-10 12:40:00,1.07373,1.07382,1.07357,1.07368,0
2,2023-01-10 12:45:00,1.07368,1.07380,1.07351,1.07376,0
3,2023-01-10 12:50:00,1.07377,1.07389,1.07343,1.07346,0
4,2023-01-10 12:55:00,1.07345,1.07367,1.07342,1.07365,0
...,...,...,...,...,...,...
995,2023-01-13 23:30:00,1.08332,1.08333,1.08310,1.08311,0
996,2023-01-13 23:35:00,1.08311,1.08312,1.08285,1.08289,0
997,2023-01-13 23:40:00,1.08289,1.08312,1.08289,1.08303,0
998,2023-01-13 23:45:00,1.08303,1.08309,1.08298,1.08305,0


In [109]:
df_train = df[:-200]
df_test = df[-200:]

In [115]:
zig_zag_percent = 0.4

zl = ZigZagClusterLevels(peak_percent_delta=zig_zag_percent, merge_distance=None,
                         merge_percent=0.2, min_bars_between_peaks=10, peaks='All')

#peaks =  Low,High,All
zl.fit(df_train)


fig = go.Figure(data=[go.Candlestick(x=df_train['Data'],
                open=df_train['Open'],
                high=df_train['High'],
                low=df_train['Low'],
                close=df_train['Close'],
                increasing_line_color= 'green', decreasing_line_color= 'red')])

for price_j in zl.levels:
    price = price_j['price']
    fig.add_hline(y=price)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=200, label="200", step="minute", stepmode="backward"),
            dict(count=500, label="500", step="minute", stepmode="backward"),
            
            dict(step="all")
        ])
    )
)

fig.update_layout(
    autosize=False,
    width=1500,
    height=800,)

fig.show()

In [116]:
new = zl.levels

In [117]:
fig = go.Figure(data=[go.Candlestick(x=df_test['Data'],
                open=df_test['Open'],
                high=df_test['High'],
                low=df_test['Low'],
                close=df_test['Close'],
                increasing_line_color= 'green', decreasing_line_color= 'red')])

for price_j in new:
    price = price_j['price']
    fig.add_hline(y=price)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=200, label="200", step="minute", stepmode="backward"),
            dict(count=500, label="500", step="minute", stepmode="backward"),
            
            dict(step="all")
        ])
    )
)

fig.update_layout(
    autosize=False,
    width=1500,
    height=800,)

fig.show()